In [ ]:
# --- Dynamics & simulation --- 
dt_emp = 0.01 ### Choose as the time we want i.e f:[0,t] -> X with t = dt_emp*T 
tsteps = 100001 
dt = 1/tsteps 
ud = .2 * dt_emp * tsteps 

def step(xt, ut):
     return xt + dt * ut 
# --- Initialize --- 
#xt = np.random.uniform(low=0.0, high=1.0, size=(2,))
xt = np.array([.5,.5])
x_traj = np.zeros((tsteps, 2)) 
traj_counts = np.zeros_like(target_probs) 
metric_log = [] 
divergence = [] 
push = 0.01
# --- Main loop --- 
for t in range(tsteps): 
    # 1. Update trajectory counts 
    ix = np.searchsorted(x_edges, xt[0], side='right') - 1 
    iy = np.searchsorted(y_edges, xt[1], side='right') - 1 
    ix = np.clip(ix, 0, num_cells_x-1) 
    iy = np.clip(iy, 0, num_cells_y-1) 
    cell_idx = ix*num_cells_y + iy 
    traj_counts[cell_idx] += 1 
    #print(target_probs) 
    # 2. Compute control direction 
    lambda_g = traj_counts / ((t+1)) 
    #print(lambda_g) 
    sigma = 0.05 # optional influence radius for decay 
    epsilon = 1e-6
    e = target_probs - lambda_g 
    e_plus = np.maximum(0, e)
    e_minus = np.minimum(e,0)
    #print(e_plus, "plus")
    #print(e_minus, "minus") 
    vectors = centroids - xt # shape (num_cells, 2) 
    dist2 = np.sum(vectors**2, axis=1) + epsilon 
    weights = np.exp(-dist2 / sigma**2) 
    direction = np.sum(e_plus[:, np.newaxis] * weights[:, np.newaxis] * (vectors / dist2[:, np.newaxis]), axis=0) + push * np.sum(e_minus[:, np.newaxis] * weights[:, np.newaxis] * (vectors / dist2[:, np.newaxis]), axis=0) 
    ut = ud * direction / (np.linalg.norm(direction) + 1e-8) 
    # 3. Step dynamics 
    xt = step(xt, ut) 
    x_traj[t] = xt.copy() 
    # 4. Compute simple ergodic metric 
    metric = np.sum((target_probs - lambda_g)**2) 
    metric_log.append(metric) 
    # Logging every 1000 steps 
    if t % 100000 == 0 and t != 0: 
        print(f"Step {t}/{tsteps}") 
        #mu_samples = sample_from_pdf_2d(pdf, [0,1], [0,1], n_samples = x_traj[:t,:].shape[0]) 
        #KL = im.kld(x_traj[:t,:],mu_samples, approach='metric') 
        #divergence.append(KL) 